# ดึงและสร้างไฟล์ฝนสสน.รายชั่วโมง ของวันที่ต้องการ


In [1]:
import requests, os
import pandas as pd
import numpy as np

from math import radians, cos, sin, asin, sqrt

from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil import rrule

from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt  
%matplotlib inline 

## ฟังก์ชั่นดึงข้อมูลฝนจากสสน.

In [2]:
# ฟังก์ชั่นดึงข้อมูลฝนจากสสน.
#คำสั่งนี้ใช้เรียกปริมาณน้ำฝนแบบรายวันจาก URL ของสถาบันสารสนเทศทรัพยากรน้ำ (องค์การมหาชน)
def get_rainfall_hii(df_filter, start, end):
  url_source = 'https://tiservice.hii.or.th/opendata/data_catalog/hourly_rain/'
  path_output = '../2results/0rain_hourly/'

  #date_start = datetime.strptime(start + '-01', '%Y-%m-%d')
  #date_end = datetime.strptime(end + '-01', '%Y-%m-%d')

  date_start = datetime.strptime(start, '%Y-%m-%d')
  date_end = datetime.strptime(end, '%Y-%m-%d')

  condition = rrule.rrule(rrule.MONTHLY, dtstart=date_start, until=date_end)
  total_process = condition.count()
  i = 1

  for date in condition:
    print('========== Start process (' + str(i) + '/' + str(total_process) + '): ' + date.strftime('%Y-%m') + ' ==========')
    year = date.strftime('%Y')
    month = date.strftime('%m')

    path = path_output + year + '/' + month
    Path(path).mkdir(parents=True, exist_ok=True)

    for code in df_filter['code']:
      # print(code)

      url = url_source + year + '/' + year + month + '/' + code.strip() + '.csv'

      # Download and save file
      result = requests.get(url)
      if result.status_code == 200:
        file_path = Path(path + '/' + code.strip() + '.csv')
        file_path.write_bytes(result.content)
        # print('Downloaded: ' + year + '/' + month + '/' + code.strip())
      else:
        print('Failed: '  + year + '/' + month + '/' + code.strip())
    i += 1

# จัดการข้อมูลฝนสสน.

## ดึงข้อมูลฝนรายชั่วโมงของเดือนที่ต้องการ ที่ตกอยู่ในสถานีในเรดาร์ มาเก็บไว้เป็นไฟล์แยกสถานีก่อน

In [3]:
# อ่านสถานีวัดฝนที่ตกอยู่ในเรดาร์พิษณุโลก
df_pl = pd.read_csv('../3GIS_results/GaugeInBasin_radar_phs_120km_wgs84.csv')
station_pl = df_pl['code']
#station_pl

# เรียก Station ของสสน.จาก URL 
df_hii = pd.read_csv('https://tiservice.hii.or.th/opendata/data_catalog/daily_rain/station_metadata.csv')
#df_hii.head()

#เรียกข้อมูลสถานีที่อยู่ภายใต้เรดาร์พิษณุโลกจากสถานีทั้งหมดของสสน.
# Strip whitespace in Column Code
df_hii['code'] = df_hii['code'].str.strip()
# Filter the DataFrame based on the list of stations
df_filter = df_hii[df_hii['code'].isin(station_pl.str.strip())]
#df_filter


# กำหนดปีเดือนเริ่มต้นและวันที่สิ้นสุดที่ต้องการดึงข้อมูล
start = '2018-06-01'
end = '2018-09-22'

# เรียกฟังก์ชั่นดึงข้อมูลจาก hii
get_rainfall_hii(df_filter, start, end)

========== Start process (1/4): 2018-06 ==========
Failed: 2018/06/FOP056
Failed: 2018/06/FOP020
Failed: 2018/06/FOP021
Failed: 2018/06/FOP055
Failed: 2018/06/FOP022
Failed: 2018/06/KWN003
========== Start process (2/4): 2018-07 ==========
Failed: 2018/07/FOP056
Failed: 2018/07/FOP020
Failed: 2018/07/FOP021
Failed: 2018/07/FOP055
Failed: 2018/07/FOP022
Failed: 2018/07/KWN003
========== Start process (3/4): 2018-08 ==========
Failed: 2018/08/FOP056
Failed: 2018/08/FOP020
Failed: 2018/08/FOP021
Failed: 2018/08/FOP055
Failed: 2018/08/FOP022
Failed: 2018/08/KWN003
========== Start process (4/4): 2018-09 ==========
Failed: 2018/09/FOP056
Failed: 2018/09/FOP020
Failed: 2018/09/FOP021
Failed: 2018/09/FOP055
Failed: 2018/09/FOP022
Failed: 2018/09/KWN003


## จัดการฝนสสน.ที่ดาวน์โหลดมา ให้อยู่ในลิสต์เดียวกัน

In [4]:
path_output = '../2results/0rain_hourly/'
#date_start = datetime.strptime(start + '-01', '%Y-%m-%d')
#date_end = datetime.strptime(end + '-01', '%Y-%m-%d')
date_start = datetime.strptime(start, '%Y-%m-%d')
date_end = datetime.strptime(end, '%Y-%m-%d')


condition = rrule.rrule(rrule.MONTHLY, dtstart=date_start, until=date_end)

prec = [] #รวมฝนสถานีที่ดาวน์โหลดมาไว้ในลิสต์นี้

for date in condition:
  year = date.strftime('%Y')
  month = date.strftime('%m')

  target = path_output + year + '/' + month + '/'

  for filename in os.listdir(target):
    station = filename.split('.')[0]
    df = pd.read_csv(target + filename)
    df['station'] = station
    prec.append(df)

#ผลที่ได้คือ ฝนรายชั่วโมงของแต่ละสถานีในเดือนนั้น ๆ 

In [5]:
#prec[:1]

# ดึงฝนเฉพาะวันและชั่วโมงที่ต้องการมาใช้งาน

## จัดข้อมูลฝนให้อยู่ใน dataframe เดียวกัน ดึงเฉพาะคอลัมน์ที่สนใจ

In [6]:
prec_pd=pd.concat(prec).reset_index(drop=False)
# get columns using dataframe subsetting
rain_subset = prec_pd[['date','time', 'rain', 'station']]
rain_subset['full_date']=rain_subset['date'].map(str)+'-'+rain_subset['time'].map(str)

#rain_subset
#ผลลัพธ์จะได้ฝนรายชั่วโมงที่อยู่ใน dataframe เดียวกันของเดือนนั้น ๆ แล้วค่อยไปดึงเฉพาะชั่วโมงของวันที่ต้องการจาก full_date

## ดึงฝนแต่ละชั่วโมงในวันที่ต้องการ แล้วส่งออกเป็น csv ในโฟลเดอร์ผลลัพธ์

In [7]:
dt_txt = ['2018-06-01-','2018-06-02-','2018-06-03-','2018-06-04-','2018-06-05-',
         '2018-06-06-','2018-06-07-','2018-06-08-','2018-06-09-','2018-06-10-',
         '2018-07-15-','2018-07-16-','2018-07-17-','2018-07-18-','2018-07-19-',
         '2018-07-20-','2018-07-21-','2018-07-22-','2018-07-23-','2018-07-24-',
         '2018-08-13-','2018-08-14-','2018-08-15-','2018-08-16-','2018-08-17-',
         '2018-08-18-','2018-08-19-','2018-08-20-','2018-08-21-','2018-08-22-',
         '2018-09-10-','2018-09-11-','2018-09-12-','2018-09-13-','2018-09-14-',
         '2018-09-15-','2018-09-16-','2018-09-17-','2018-09-18-','2018-09-19-',
         '2018-09-20-','2018-09-21-'] #ฝนรายชั่วโมงในวันที่ต้องการ
#dt_txt = ['2018-07-21-'] #ฝนรายชั่วโมงในวันที่ต้องการ
hr_txt = rain_subset['time'].unique()

out_rain_hr_path = '../2results/0_outp_rain_hr/'

for d in dt_txt:
    #print
    for h in hr_txt:
        dh=d+h
        dt=datetime.strptime(dh, '%Y-%m-%d-%H:%M')
        print(dh+'>>>')
        rain_ =rain_subset[rain_subset['full_date']==str(dh)]
        
        # qc 3 ค่า >> nan '-' -999.0
        df_rain_pl=rain_.merge(df_filter,left_on='station', right_on='code')        
        df_ = df_rain_pl.drop(df_rain_pl[df_rain_pl['rain'] == '-'].index)
        df_ = df_.drop(df_[df_['rain'] == -999.0].index)
        df_ = df_.dropna().reset_index(drop=True)              
        
        #ส่งออกฝนรายชั่วโมง csv
        file_hr = dt.strftime('%Y')+dt.strftime('%m')+dt.strftime('%d')+dt.strftime('%H')+'.csv'
        df_.to_csv(out_rain_hr_path + file_hr, encoding='utf-8-sig')


2018-06-01-00:00>>>
2018-06-01-01:00>>>
2018-06-01-02:00>>>
2018-06-01-03:00>>>
2018-06-01-04:00>>>
2018-06-01-05:00>>>
2018-06-01-06:00>>>
2018-06-01-07:00>>>
2018-06-01-08:00>>>
2018-06-01-09:00>>>
2018-06-01-10:00>>>
2018-06-01-11:00>>>
2018-06-01-12:00>>>
2018-06-01-13:00>>>
2018-06-01-14:00>>>
2018-06-01-15:00>>>
2018-06-01-16:00>>>
2018-06-01-17:00>>>
2018-06-01-18:00>>>
2018-06-01-19:00>>>
2018-06-01-20:00>>>
2018-06-01-21:00>>>
2018-06-01-22:00>>>
2018-06-01-23:00>>>
2018-06-02-00:00>>>
2018-06-02-01:00>>>
2018-06-02-02:00>>>
2018-06-02-03:00>>>
2018-06-02-04:00>>>
2018-06-02-05:00>>>
2018-06-02-06:00>>>
2018-06-02-07:00>>>
2018-06-02-08:00>>>
2018-06-02-09:00>>>
2018-06-02-10:00>>>
2018-06-02-11:00>>>
2018-06-02-12:00>>>
2018-06-02-13:00>>>
2018-06-02-14:00>>>
2018-06-02-15:00>>>
2018-06-02-16:00>>>
2018-06-02-17:00>>>
2018-06-02-18:00>>>
2018-06-02-19:00>>>
2018-06-02-20:00>>>
2018-06-02-21:00>>>
2018-06-02-22:00>>>
2018-06-02-23:00>>>
2018-06-03-00:00>>>
2018-06-03-01:00>>>


In [8]:
# ตรวจสอบโฟลเดอร์ที่เก็บไฟล์ csv ที่สร้างไว้
# ต่อจะสร้างเรดาร์ dbz รายชั่วโมงแบบ cappi/ppi แล้วนำค่าฝนสสน.มาสกัด เพื่อสร้างแบบจำลอง regression ML ต่อไป